In [1]:
import numpy as np
import pandas as pd
import operator
import itertools
from itertools import combinations, chain

In [2]:
data_main= pd.read_csv(r"C:\Users\diyas\Documents\Recommendation_system\combinedFinalMasterDataset_updatedSoftwareTools.csv") 
data_main['TierLevel']=data_main['TierLevel'].replace(to_replace ="1.0",value ="1")
data_main['TierLevel']=data_main['TierLevel'].replace(to_replace ="2.0",value ="2")
data_main['TierLevel']=data_main['TierLevel'].replace(to_replace ="3.0",value ="3")

In [43]:
"""var='ScholarshipsNLP'
var='AwardsNLP'
var='PublicationNLP'
var='WorkshopsDomain'
var='ResearchDomain'
var='InternshipProjectDomain'
var='ProgLanguages'
var='SoftwareTools'
var='ProjectDetailDomain'
var='LanguagesNLP'
var='ExternalCertificatesDomain'
var='WorkshopsOrg'
var='WorkshopsDomain'
var='VolunteeringWork"""

"var='ScholarshipsNLP'\nvar='AwardsNLP'\nvar='PublicationNLP'\nvar='WorkshopsDomain'\nvar='ResearchDomain'\nvar='InternshipProjectDomain'\nvar='ProgLanguages'\nvar='SoftwareTools'\nvar='ProjectDetailDomain'\nvar='LanguagesNLP'\nvar='ExternalCertificatesDomain'\nvar='WorkshopsOrg'\nvar='WorkshopsDomain'\nvar='VolunteeringWork"

In [48]:
data_main.columns

Index(['Unnamed: 0', 'USN', '10thPercentage', '12thPercentage', 'AwardsNLP',
       'Branch', 'CGPA', 'CTC', 'CoCurricularActivities', 'CompanyNameNLP',
       'CoreCourseGrade', 'CoreCourseName', 'CourseGrade', 'CourseName',
       'ElectiveCourseGrade', 'ElectiveCourseName', 'EmploymentType',
       'ExternalCertificatesDomain', 'ExternalCertificatesKey',
       'GeneralSkills', 'GitHubLink', 'InternshipCompany',
       'InternshipProjectDomain', 'LanguagesNLP', 'LinkedInLink',
       'MinorAttended', 'Name', 'NoofInternships', 'NoofProjects',
       'NumberOfOffers', 'OtherDetails', 'ProgLanguages',
       'ProjectDetailDomain', 'PublicationNLP', 'ResearchDomain',
       'ScholarshipsNLP', 'SoftwareTools', 'Stipend(K)', 'TierLevel',
       'VolunteeringWork', 'WorkshopsDomain', 'WorkshopsOrg'],
      dtype='object')

In [19]:
#selection of dataset
var='SoftwareTools'
#data_main=data_main.apply(lambda x: x.astype(str).str.upper())

In [20]:
#GENERAL CODE 

#Consider alpha(between 1 and 0) 0.1
alpha= 0.1
ls=4

data= data_main[[var,'TierLevel']]
#print(data)
data = data.dropna()
data = data[data['TierLevel'] == '1'] 
data[var]=data[var].str.split('##')

print("FOR ",var)
print("LS = ",ls)
print("\n")
#finding frequency of domains
domain_count={}
for v,k in data[var].iteritems():
    for i in k:
        if i in domain_count:
            domain_count[i]=domain_count[i]+1
        else:
            domain_count[i]=1
#calulate MIS for each itemset

#GIVE PARAMETERS

#Step 1: find SD
#Lambda(mean)
lamb = 0
for val in domain_count.values(): 
    lamb += val 
lamb = lamb / len(domain_count) 


def mis(domain_count,ls,alpha):
    sd=lamb*(1-alpha)
    mis=[]
    misdict={}
    mis_val={}
    for k, v in domain_count.items():
        m=domain_count[k]-sd
        if(m<ls):
            mis.append(ls)
            misdict[domain_count[k]]=ls
            mis_val[k]=ls
        else:
            mis.append(m)
            misdict[domain_count[k]]=m
            mis_val[k]=m
    #print(mis_val)
    return mis_val

mis_val=mis(domain_count,ls,alpha)

mis_min2=min(mis_val.values())
L_2={}
for k, v in domain_count.items():
        if(domain_count[k]>=mis_val[k]):
            L_2[k]=mis_val[k]

#Step 2
#Sort L
temp= sorted(L_2.items(), key=operator.itemgetter(0))
L_21=sorted(temp, key=operator.itemgetter(1), reverse=True)
#2 ITEMSETS 
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n)))   
n = 2
C2=findsubsets(L_21, n)

#count for each candidate itemset
count={}
for i,v in C2:
    for index, row in data.iterrows():
        t=""
        if(i[0] in row[var] and v[0] in row[var]):
            t=i[0]+","+v[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

                #checking criteria for count
L_12={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_12[k]=count[k]
            
#FOR TIER 1, SORTED 2 ITEMSETS 
temp= sorted(L_12.items(), key=operator.itemgetter(0))
L_12_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("2 itemset")
print(L_12_sorted)

#3 ITEMSETS
n = 3
C3=findsubsets(L_21, n)

count={}
for a,b,c in C3:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row[var] and b[0] in row[var] and c[0] in row[var]):
            t=a[0]+","+b[0]+','+c[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1
                
L_3={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_3[k]=count[k]
            
#FOR TIER 1, SORTED 3 ITEMSETS 
temp= sorted(L_3.items(), key=operator.itemgetter(0))
L_3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("\n")
print("3 itemset")
print(L_3_sorted)

#4 ITEMSETS 
n = 4
C4=findsubsets(L_21, n)

count={}
for a,b,c,d in C4:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row[var] and b[0] in row[var] and c[0] in row[var] and d[0] in row[var]):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L_4={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_4[k]=count[k]
            
#FOR TIER 1, SORTED 4 ITEMSETS 
temp= sorted(L_4.items(), key=operator.itemgetter(0))
L_4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("\n")
print("4 ITEMSET")
print(L_4_sorted)

FOR  SoftwareTools
LS =  4


2 itemset
[]


3 itemset
[]


4 ITEMSET
[]


In [ ]:
class Apriori:
    def __init__(self, masterData):
        self.masterData = masterData
        
    def findsubsets(self,s, n):
        return list(map(set, itertools.combinations(s, n)))
    
    def generateItemsets(self,data):
        
        
    def subsetData(self,data_main,var):
        data= data_main[[var,'TierLevel']]
        #print(data)
        data = data.dropna()
        data = data[data['TierLevel'] == '1'] 
        data[var]=data[var].str.split('##')
        return data
        
    def findDomainCount(self,var,data,ls):
        print("FOR ",var)
        print("LS = ",ls)
        print("\n")
        #finding frequency of domains
        domain_count={}
        for v,k in data[var].iteritems():
            for i in k:
                if i in domain_count:
                    domain_count[i]=domain_count[i]+1
                else:
                    domain_count[i]=1
        return domian_count
                    
    def findLambda(domain_count):
        lamb = 0
        for val in domain_count.values(): 
            lamb += val 
        lamb = lamb / len(domain_count) 
        return lamb

    def mis(domain_count,ls,alpha):
        sd=lamb*(1-alpha)
        mis=[]
        misdict={}
        mis_val={}
        for k, v in domain_count.items():
            m=domain_count[k]-sd
            if(m<ls):
                mis.append(ls)
                misdict[domain_count[k]]=ls
                mis_val[k]=ls
            else:
                mis.append(m)
                misdict[domain_count[k]]=m
                mis_val[k]=m
        #print(mis_val)
        return mis_val
    
    def filterMIS(mis_val,domain_count):
        mis_min2=min(mis_val.values())
        L_2={}
        for k, v in domain_count.items():
                if(domain_count[k]>=mis_val[k]):
                    L_2[k]=mis_val[k]


In [3]:
#Preprocessing for software tools
var='SoftwareTools'
data= data_main[[var,'TierLevel']]
#print(data)
data = data.dropna()
data = data[data['TierLevel'] == 1.0] 
data[var]=data[var].str.split('##')

In [4]:
for i,v in data.iterrows():
    #print(data['SoftwareTools'][i])
    l=[]
    for k in v['SoftwareTools']:
        if(k!=''):
            t=k.rstrip("\n")
            h=t.strip()
            #print(h)
            l.append(h.lower())
    data['SoftwareTools'][i]=l

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [5]:


    #GENERAL CODE 

#Consider alpha(between 1 and 0) 0.1
alpha= 0.1
ls=2

print("FOR ",var)
print("LS = ",ls)
print("\n")
#finding frequency of domains
domain_count={}
for v,k in data[var].iteritems():
    for i in k:
        if i in domain_count:
            domain_count[i]=domain_count[i]+1
        else:
            domain_count[i]=1
#calulate MIS for each itemset

#GIVE PARAMETERS

#Step 1: find SD
#Lambda(mean)
lamb = 0
for val in domain_count.values(): 
    lamb += val 
lamb = lamb / len(domain_count) 


def mis(domain_count,ls,alpha):
    sd=lamb*(1-alpha)
    mis=[]
    misdict={}
    mis_val={}
    for k, v in domain_count.items():
        m=domain_count[k]-sd
        if(m<ls):
            mis.append(ls)
            misdict[domain_count[k]]=ls
            mis_val[k]=ls
        else:
            mis.append(m)
            misdict[domain_count[k]]=m
            mis_val[k]=m
    #print(mis_val)
    return mis_val

mis_val=mis(domain_count,ls,alpha)

mis_min2=min(mis_val.values())
L_2={}
for k, v in domain_count.items():
        if(domain_count[k]>=mis_val[k]):
            L_2[k]=mis_val[k]

#Step 2
#Sort L
temp= sorted(L_2.items(), key=operator.itemgetter(0))
L_21=sorted(temp, key=operator.itemgetter(1), reverse=True)
#2 ITEMSETS 
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n)))   
n = 2
C2=findsubsets(L_21, n)

#count for each candidate itemset
count={}
for i,v in C2:
    for index, row in data.iterrows():
        t=""
        if(i[0] in row[var] and v[0] in row[var]):
            t=i[0]+","+v[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

                #checking criteria for count
L_12={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_12[k]=count[k]
            
#FOR TIER 1, SORTED 2 ITEMSETS 
temp= sorted(L_12.items(), key=operator.itemgetter(0))
L_12_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("2 itemset")
print(L_12_sorted)

#3 ITEMSETS
n = 3
C3=findsubsets(L_21, n)

count={}
for a,b,c in C3:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row[var] and b[0] in row[var] and c[0] in row[var]):
            t=a[0]+","+b[0]+','+c[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1
                
L_3={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_3[k]=count[k]
            
#FOR TIER 1, SORTED 3 ITEMSETS 
temp= sorted(L_3.items(), key=operator.itemgetter(0))
L_3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("\n")
print("3 itemset")
print(L_3_sorted)

#4 ITEMSETS 
n = 4
C4=findsubsets(L_21, n)

count={}
for a,b,c,d in C4:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row[var] and b[0] in row[var] and c[0] in row[var] and d[0] in row[var]):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L_4={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_4[k]=count[k]
            
#FOR TIER 1, SORTED 4 ITEMSETS 
temp= sorted(L_4.items(), key=operator.itemgetter(0))
L_4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("\n")
print("4 ITEMSET")
print(L_4_sorted)

FOR  SoftwareTools
LS =  2


2 itemset
[('android studio,aws', 6)]


KeyboardInterrupt: 

In [7]:
#SOFTWARE TOOLS BASED GROUPING

#Adobe:
l1=['adobe animate','adobe illustrator', 'adobe photoshop']
#Cloud Computing and Big Data
l2=['aws','docker','google cloud','spark','hadoop','hive','hbase','scala','ngnix','jenkins','elasticsearch']
#Python tools
l3=['numpy','flask','tkinter','spyder','django','junit','jupyter notebook','simpy','pytorch','pygame']
#Microsoft Tools
l4=['ms access','ms office','ms powerpoint','ms word']
#DBMS TOOLS
l5=['sql','mysql','sqlite','mongodb','postgresql']
#Machine Learning Tools
l6=['keras','tensorflow','tflearn','matlab']
#Computer Network
l7=['cisco packet tracer','claynet','wireshark']

In [8]:
var='SoftwareTools'
data= data_main[[var,'TierLevel']]
#print(data)
data = data.dropna()
data = data[data['TierLevel'] == 1.0] 
data[var]=data[var].str.split('##')
for i,v in data.iterrows():
    #print(data['SoftwareTools'][i])
    l=[]
    for k in v['SoftwareTools']:
        if(k!=''):
            t=k.rstrip("\n")
            h=t.strip()
            #print(h)
            l.append(h.lower())
    data['SoftwareTools'][i]=l

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [9]:
for i,v in data.iterrows():
    #print(data['SoftwareTools'][i])
    l=[]
    for k in v['SoftwareTools']:
        if(k in l1):
            l.append("Adobe")
        if(k in l2):
            l.append("Cloud Computing and Big Data")
        if(k in l3):
            l.append("Python tools")
        if(k in l4):
            l.append("Microsoft Tools")
        if(k in l5):
            l.append("DBMS TOOLS")
        if(k in l6):
            l.append("Machine Learning Tools")
        if(k in l7):
            l.append("Computer Network")
    data['SoftwareTools'][i]=l

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:


    #GENERAL CODE 

#Consider alpha(between 1 and 0) 0.1
alpha= 0.1
ls=2

print("FOR ",var)
print("LS = ",ls)
print("\n")
#finding frequency of domains
domain_count={}
for v,k in data[var].iteritems():
    for i in k:
        if i in domain_count:
            domain_count[i]=domain_count[i]+1
        else:
            domain_count[i]=1
#calulate MIS for each itemset

#GIVE PARAMETERS

#Step 1: find SD
#Lambda(mean)
lamb = 0
for val in domain_count.values(): 
    lamb += val 
lamb = lamb / len(domain_count) 


def mis(domain_count,ls,alpha):
    sd=lamb*(1-alpha)
    mis=[]
    misdict={}
    mis_val={}
    for k, v in domain_count.items():
        m=domain_count[k]-sd
        if(m<ls):
            mis.append(ls)
            misdict[domain_count[k]]=ls
            mis_val[k]=ls
        else:
            mis.append(m)
            misdict[domain_count[k]]=m
            mis_val[k]=m
    #print(mis_val)
    return mis_val

mis_val=mis(domain_count,ls,alpha)

mis_min2=min(mis_val.values())
L_2={}
for k, v in domain_count.items():
        if(domain_count[k]>=mis_val[k]):
            L_2[k]=mis_val[k]

#Step 2
#Sort L
temp= sorted(L_2.items(), key=operator.itemgetter(0))
L_21=sorted(temp, key=operator.itemgetter(1), reverse=True)
#2 ITEMSETS 
def findsubsets(s, n):
    return list(map(set, itertools.combinations(s, n)))   
n = 2
C2=findsubsets(L_21, n)

#count for each candidate itemset
count={}
for i,v in C2:
    for index, row in data.iterrows():
        t=""
        if(i[0] in row[var] and v[0] in row[var]):
            t=i[0]+","+v[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

                #checking criteria for count
L_12={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_12[k]=count[k]
            
#FOR TIER 1, SORTED 2 ITEMSETS 
temp= sorted(L_12.items(), key=operator.itemgetter(0))
L_12_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("2 itemset")
print(L_12_sorted)

#3 ITEMSETS
n = 3
C3=findsubsets(L_21, n)

count={}
for a,b,c in C3:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row[var] and b[0] in row[var] and c[0] in row[var]):
            t=a[0]+","+b[0]+','+c[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1
                
L_3={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_3[k]=count[k]
            
#FOR TIER 1, SORTED 3 ITEMSETS 
temp= sorted(L_3.items(), key=operator.itemgetter(0))
L_3_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("\n")
print("3 itemset")
print(L_3_sorted)

#4 ITEMSETS 
n = 4
C4=findsubsets(L_21, n)

count={}
for a,b,c,d in C4:
    for index, row in data.iterrows():
        t=""
        if(a[0] in row[var] and b[0] in row[var] and c[0] in row[var] and d[0] in row[var]):
            t=a[0]+","+b[0]+','+c[0]+','+d[0]
            if(t in count):
                count[t]=count[t]+1
            else:
                count[t]=1

L_4={}
for k, v in count.items():
        m=count[k]*100/272
        if(m>mis_min2):
            L_4[k]=count[k]
            
#FOR TIER 1, SORTED 4 ITEMSETS 
temp= sorted(L_4.items(), key=operator.itemgetter(0))
L_4_sorted=sorted(temp, key=operator.itemgetter(1), reverse=True)
print("\n")
print("4 ITEMSET")
print(L_4_sorted)

FOR  SoftwareTools
LS =  2


2 itemset
[('DBMS TOOLS,Cloud Computing and Big Data', 10)]


3 itemset
[]


4 ITEMSET
[]
